In [2]:
import torch
from neighborhoods import LocalNeighborhood

/home/megvii/anaconda3/envs/paul-py3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coordinates=['index_distance']
config = None
nei_layer = LocalNeighborhood(
    config, Kmax=14, coordinates=coordinates, self_neighborhood=True)

bs = 2
s1 = 256
h = 32
indices = torch.arange(s1).repeat(bs, 1).unsqueeze(dim=-1)
feat1 = torch.rand(bs, s1, h)
feat2 = torch.rand(bs, s1, h)
inputs = [indices, feat1, feat2]
output = nei_layer(inputs)

neighbor_coordinates size=1


In [3]:
for it in output:
    for iit in it:
        print(iit.shape)

torch.Size([2, 256, 14, 1])
torch.Size([256, 14, 32])
torch.Size([256, 14, 32])
torch.Size([256, 14, 32])
torch.Size([256, 14, 32])


In [10]:
output[0][0].shape

torch.Size([2, 256, 14, 1])

In [14]:
import torch.nn as nn
rnn = nn.RNN(10, 20, 2)
input = torch.randn(5, 3, 10)

output, hn = rnn(input)
output.shape, hn.shape

(torch.Size([5, 3, 20]), torch.Size([2, 3, 20]))

In [15]:
x1 = torch.randn(5, 3, 10)
x2 = torch.randn(5, 3, 10)
torch.cat([x1, x2],dim=-1).shape

torch.Size([5, 3, 20])

In [18]:
import torch.nn as nn

class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=False):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):

        if len(x.size()) <= 2:
            return self.module(x)

        # Squash samples and timesteps into a single axis
        x_reshape = x.contiguous().view(-1, x.size(-1))  # (samples * timesteps, input_size)

        y = self.module(x_reshape)

        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), -1, y.size(-1))  # (samples, timesteps, output_size)
        else:
            y = y.view(-1, x.size(1), y.size(-1))  # (timesteps, samples, output_size)

        return y

In [20]:
# torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
dense = nn.Linear(20, 32, bias=False)
seq_net = TimeDistributed(dense, batch_first=True)

x1 = torch.randn(5, 3, 20)
y = seq_net(x1)
y.shape

torch.Size([5, 3, 32])

In [26]:
from modeling.lib import GlobalGraph

inputs = torch.rand(2, 10, 64)
mask = torch.ones([2,10,10])

gnet = GlobalGraph(64)
output = gnet(inputs, mask)
output.shape

torch.Size([2, 10, 64])

In [36]:
# tf.gather_nd(points, triplets[:, :, 1:2], batch_dims=1)
points = torch.rand(2,10,3)
triplets = torch.randint(0, 10, (2,5,3))
offsets = (torch.arange(2) * 10).reshape(2,1,1)

indices = triplets[:,:,1:2]
print(indices)
indices = indices + offsets
print(indices)


tensor([[[2],
         [0],
         [5],
         [5],
         [2]],

        [[3],
         [0],
         [1],
         [3],
         [4]]])
tensor([[[ 2],
         [ 0],
         [ 5],
         [ 5],
         [ 2]],

        [[13],
         [10],
         [11],
         [13],
         [14]]])


In [42]:
import numpy as np
batch_dim =3
batch_size = 5
params = torch.randn(batch_size, batch_size, batch_size, 4, 4, 4, 32)
indices = torch.randint(4, (batch_size, batch_size, batch_size, 12, 3))

batch_dims = params.size()[:batch_dim]  # [b1, ..., bn]
batch_size = np.cumprod(list(batch_dims))[-1]  # b1 * ... * bn
c_dim = params.size()[-1]  # c
grid_dims = params.size()[batch_dim:-1]  # [g1, ..., gm]
n_indices = indices.size(-2)  # x
n_pos = indices.size(-1)  # m

# reshape leadning batch dims to a single batch dim
params = params.reshape(batch_size, *grid_dims, c_dim)
indices = indices.reshape(batch_size, n_indices, n_pos)

batch_enumeration = torch.arange(batch_size).unsqueeze(1)
gather_dims = [indices[:, :, i] for i in range(len(grid_dims))]
gather_dims.insert(0, batch_enumeration)




torch.Size([125, 4, 4, 4, 32])

In [64]:
params = torch.rand(2, 5, 2)
print(params)
batch_dim = torch.arange(2).unsqueeze(1).unsqueeze(1)
dim2 = torch.tensor([ [[0,1,2], [1,2,3]], [[2,3,4], [2,3,4]] ])
dims = [batch_dim, dim2]
params[dims], params[dims].shape

tensor([[[0.6688, 0.5033],
         [0.9680, 0.8551],
         [0.5141, 0.1596],
         [0.1239, 0.5980],
         [0.9618, 0.2434]],

        [[0.4828, 0.1921],
         [0.5933, 0.6344],
         [0.5788, 0.8869],
         [0.3521, 0.6423],
         [0.8749, 0.6521]]])


(tensor([[[[0.6688, 0.5033],
           [0.9680, 0.8551],
           [0.5141, 0.1596]],
 
          [[0.9680, 0.8551],
           [0.5141, 0.1596],
           [0.1239, 0.5980]]],
 
 
         [[[0.5788, 0.8869],
           [0.3521, 0.6423],
           [0.8749, 0.6521]],
 
          [[0.5788, 0.8869],
           [0.3521, 0.6423],
           [0.8749, 0.6521]]]]),
 torch.Size([2, 2, 3, 2]))

In [77]:
sz1 = torch.Size([5 for _ in range(2)])
sz2 = torch.Size([1 for _ in range(2)])
x = torch.rand(25,1)
x.reshape(*sz1, *sz2, 1).shape

torch.Size([5, 5, 1, 1, 1])

In [3]:
import torch
from neighborhoods import gather_nd_torch_ex

In [5]:
params = torch.rand(2,4,4,3)
indices = torch.randint(4,(2,6,3,2))
out = gather_nd_torch_ex(params,indices,1)
out.shape

torch.Size([2, 6, 3, 3])

In [14]:
x = torch.tensor([[1,0,0], [1,0,0]]) # (2,3)
x = x.expand(4,2,3)
y = torch.tensor([[0,-1,0], [1,0,0], [0,0,1]]) # (3,3)
y = y.expand(2,3,3)

torch.tensordot(x,y, ([-1], [-1])).shape

torch.Size([4, 2, 2, 3])